# Setup enviorment

In [1]:
# Data reading in Dataframe format and data preprocessing
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Linear algebra operations
import numpy as np

# Machine learning models and preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

# Epiweek
from epiweeks import Week, Year

# Date
from datetime import date as convert_to_date

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
embeddings = 'Embeddings/embeddings_vae_1024features.csv'
labels = 'Tabular_data/Label_CSV_All_Municipality.csv'
Municipalities = ['Medellín', 'Cali', 'Villavicencio', 'Cúcuta', 'Ibagué']

# Read Data

In [4]:
def epiweek_from_date(image_date):
    date = image_date.split('-')
    
    # Get year as int
    year = ''.join(filter(str.isdigit, date[0]))
    year = int(year)
    
    # Get month as int
    month = ''.join(filter(str.isdigit, date[1]))
    month = int(month)
    
    # Get day as int
    day = ''.join(filter(str.isdigit, date[2]))
    day = int(day)
    
    # Get epiweek:
    date = convert_to_date(year, month, day)
    epiweek = str(Week.fromdate(date))
    epiweek = int(epiweek)
    
    return epiweek

### 1. Features

In [5]:
def read_features(path, Municipality = None):
    print(Municipality)
    df = pd.read_csv(path)
    #df.Date = pd.to_datetime(df.Date)
    
    if Municipality:
        print('Obtaining dataframe for the city of Medellin only...')
        df = df[df['Municipality Code'] == Municipality]
        
    df.Date = df.Date.apply(epiweek_from_date)
    
    df = df.sort_values(by=['Date'])
    
    df = df.set_index('Date')
    
    if Municipality:
        df.drop(columns=['Municipality Code'], inplace=True)

    
    df.index.name = None
    return df

In [6]:
features_df = [read_features(path=embeddings, Municipality=municipality) for municipality in Municipalities]
features_df[0]

Medellín
Obtaining dataframe for the city of Medellin only...
Cali
Obtaining dataframe for the city of Medellin only...
Villavicencio
Obtaining dataframe for the city of Medellin only...
Cúcuta
Obtaining dataframe for the city of Medellin only...
Ibagué
Obtaining dataframe for the city of Medellin only...


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
201544,-0.046489,-1.407201,-1.148883,0.836687,-0.190778,1.802231,0.517497,-1.146697,0.119171,1.544828,...,-0.678144,-0.036498,-0.303533,0.248490,-0.811357,-0.832925,-0.739305,-0.295200,-0.038603,0.485674
201545,0.327904,-0.701594,0.306197,-0.796750,-1.037395,0.857556,0.167456,1.642044,-0.473234,0.181270,...,-0.897844,-0.661285,-1.146673,-1.234707,-0.184185,-0.140669,-2.367193,-0.463623,-0.668936,-0.233073
201546,-1.634988,-0.882196,0.291420,-2.686286,-0.684004,-0.407912,1.827255,0.327352,0.987357,0.781570,...,0.906685,1.372330,-0.333867,0.728862,-0.005498,-0.743434,0.597319,0.195301,-0.850010,0.492581
201547,1.789035,0.854698,-2.904163,-1.008464,-0.165688,0.597984,0.142369,0.164123,1.069377,0.236612,...,0.254610,-0.536157,-0.424962,0.685125,-0.029501,0.095485,0.379560,-0.297148,-1.211405,0.443391
201548,-0.983180,0.145126,0.720849,1.524544,0.620878,-1.807810,0.340521,0.781281,-0.395229,-0.769379,...,1.562922,0.585608,-0.675990,0.232964,0.788353,0.147064,0.094748,-0.206415,1.170020,0.276934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201848,-0.177821,0.800357,0.158949,2.573438,1.615337,-0.998008,1.096824,-0.023870,-0.667530,-1.929526,...,2.209477,0.241274,-1.311355,0.955204,0.006918,0.715542,-0.870650,-0.205803,0.835786,1.692061
201849,-0.306444,1.308680,1.067536,-0.931025,-1.046062,2.075529,-0.501223,-2.267342,0.095569,-0.845871,...,-2.447275,0.293684,-1.791375,-0.086890,0.917567,0.631528,-2.004170,0.267849,1.151415,-1.051147
201850,-0.151392,0.071449,0.356428,0.259774,-0.383608,1.065443,0.450759,0.322183,-1.307717,-0.181305,...,1.033319,0.951084,0.638319,-0.277678,0.032450,-0.329121,-0.322632,-1.898407,-0.602611,-0.267056
201851,-0.793142,-0.539722,-1.165816,-0.199017,2.244849,0.917043,-0.467799,0.353362,1.053764,0.443968,...,0.882802,-0.827243,-0.127070,-1.308614,-1.632518,1.059085,1.047949,-2.762428,-0.955669,0.913588


### 2. Labels

In [7]:
def get_epiweek(name):
    
    # Get week
    week = name.split('/')[1]
    week = week.replace('w','')
    week = int(week)
    
    # Year
    year = name.split('/')[0]
    year = int(year)
    
    epiweek = Week(year, week)
    
    epiweek = str(epiweek)
    epiweek = int(epiweek)

    return epiweek

In [8]:
def read_labels(path, Municipality = None):
    df = pd.read_csv(path)
    if df.shape[1] > 678:
        df = pd.concat([df[['Municipality code', 'Municipality']], df.iloc[:,-676:]], axis=1)
        cols = df.iloc[:, 2:].columns
        new_cols = df.iloc[:, 2:].columns.to_series().apply(get_epiweek)
        df = df.rename(columns=dict(zip(cols, new_cols))) 
        
    if 'Label_CSV_All_Municipality' in path:
        # Get Columns
        df = df[['epiweek', 'Municipality code', 'Municipality', 'final_cases_label']]
        
        # change epiweek format
        df.epiweek = df.epiweek.apply(get_epiweek)
        
        # Remove duplicates
        df = df[df.duplicated(['epiweek','Municipality code','Municipality']) == False]
        
        # Replace Increase, decrease, stable to numerical:
        """
        - Stable = 0
        - Increased = 1 
        - Decreased = 2
        """
        df.final_cases_label = df.final_cases_label.replace({'Stable': 0, 'Increased': 1, 'Decreased': 2})
        
        # Create table
        df = df.pivot(index=['Municipality code', 'Municipality'], columns='epiweek', values='final_cases_label')

        # Reset Index:
        df = df.reset_index()
    
    if Municipality:
        df = df[df['Municipality'] == Municipality]
        df.drop(columns=['Municipality code'], inplace=True)
        df.rename(columns={'Municipality': 'Municipality Code'}, inplace=True)
    
        df = df.set_index('Municipality Code')
        df = df.T

        df.columns.name = None
        df.index.name = None
        
        df.columns = ['Labels']
    
    return df

In [9]:
labels_df = [read_labels(path=labels, Municipality=municipality) for municipality in Municipalities]
labels_df = [pd.get_dummies(df['Labels']) for df in labels_df]

#labels_df[0]

# Data preparation

In [10]:
n_labels = labels_df[0].shape[1]

In [11]:
# Merge the two dataframes based on the date values
dengue_df = [features_df[i].merge(labels_df[i], how='inner', left_index=True, right_index=True) for i in range(len(labels_df))]
dengue_df[1]

,0,1,2,3,4,5,6,7,8,9,...,1017,1018,1019,1020,1021,1022,1023,0,1,2
201601,0.530339,0.836634,-0.189898,-1.399774,-0.163630,0.386881,-1.431141,-0.421500,1.632680,-0.550919,...,-1.274454,2.191109,-0.452808,0.267760,-0.081853,0.739175,-1.778660,1,0,0
201602,-1.286659,0.890329,1.199037,0.372843,0.956241,0.429808,1.541668,0.097834,0.138674,0.174439,...,1.606132,0.433973,0.202521,0.571386,0.327917,-0.927536,0.618543,0,1,0
201603,-0.611733,0.066734,0.309593,0.428333,-0.289946,0.945357,-0.438024,-0.933955,-1.101725,1.226060,...,1.281231,0.149778,0.342411,0.264272,-2.013417,-1.171865,0.112934,0,1,0
201604,-0.651634,0.138970,-0.982205,-0.246530,0.202211,0.413163,-1.000387,0.852654,-1.085399,-0.267373,...,0.275477,-1.363997,0.995036,1.119314,-0.430456,-1.419792,-0.738468,0,1,0
201605,-0.810654,0.799930,0.921009,0.598906,0.533667,0.162489,1.957477,0.017144,1.824350,0.546452,...,0.917176,0.433738,-2.231180,1.458560,-0.659694,-2.016434,0.708075,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201848,-0.300972,-0.811120,1.164669,0.785566,-0.440794,0.091074,-1.021946,0.106130,-0.779507,-0.038890,...,0.607345,-0.388624,-0.165844,0.311355,0.553579,-0.664937,1.892388,1,0,0
201849,-0.835302,-2.311434,-1.303870,-2.390785,1.215876,-1.202574,-0.601447,0.536091,0.370814,0.301591,...,0.237816,-1.686966,1.093373,0.287418,-0.263428,0.020032,0.868360,1,0,0
201850,-0.347228,-1.308092,-0.842854,0.845182,-0.553490,0.217280,-1.048339,-0.836241,-0.572967,-1.366528,...,0.359383,-0.441211,-0.581947,-0.324204,-0.077946,-0.713509,2.008128,1,0,0
201851,0.579590,-0.609673,-1.108059,1.371473,-0.296056,1.981669,0.858762,1.435440,-0.294704,-1.007653,...,-1.231365,1.318080,0.479897,0.177258,-0.111455,-0.554494,-1.473663,1,0,0


### Train Test split

In [12]:
def train_test_split(df, train_percentage = 80):
    # We need a sequence so we can't split randomly
    # To divide into Train and test we have to calculate the train percentage of the dataset:
    size = df.shape[0]
    split = int(size*(train_percentage/100))
    
    """ Train """
    # We will train with 1st percentage % of data and test with the rest
    train_df = df.iloc[:split,:] ## percentage % train
    
    """ Test """
    test_df = df.iloc[split:,:] # 100 - percentage % test
    
    print(f'The train shape is: {train_df.shape}')
    print(f'The test shape is: {test_df.shape}')
    
    return train_df, test_df

In [13]:
train_df = []
test_df = []

for i in range(len(dengue_df)):
    train_df_aux, test_df_aux = train_test_split(dengue_df[i], train_percentage = 80)
    train_df.append(train_df_aux)
    test_df.append(test_df_aux)
#test_df

The train shape is: (124, 1027)
The test shape is: (32, 1027)
The train shape is: (124, 1027)
The test shape is: (32, 1027)
The train shape is: (124, 1027)
The test shape is: (32, 1027)
The train shape is: (124, 1027)
The test shape is: (32, 1027)
The train shape is: (124, 1027)
The test shape is: (32, 1027)


### Normalize features

In [14]:
# Normalize train data and create the scaler
def normalize_train_features(df, feature_range=(-1, 1), n_labels=None):
    
    if n_labels:
        n_features = df.shape[1] - n_labels
    
    scalers = {}
    # For each column in the dataframe
    for i, column in enumerate(df.columns):
        if n_labels:
            if i >= n_features:
                break
        # Get values of the column
        values = df[column].values.reshape(-1,1)
        # Generate a new scaler
        scaler = MinMaxScaler(feature_range=feature_range)
        # Fit the scaler just for that column
        scaled_column = scaler.fit_transform(values)
        # Add the scaled column to the dataframe
        scaled_column = np.reshape(scaled_column, len(scaled_column))
        df[column] = scaled_column
        
        # Save the scaler of the column
        scalers['scaler_' + column] = scaler
        
    print(f' Min values are: ')
    print(df.min())
    print(f' Max values are: ')
    print(df.max())
        
    return df, scalers


""" If you want to use the same scaler used in train, you can use this function"""
def normalize_test_features(df, scalers=None, n_labels=None):
    
    if not scalers:
        raise TypeError("You should provide a list of scalers.")
    
    if n_labels:
        n_features = df.shape[1] - n_labels
    
    for i, column in enumerate(df.columns):
        if n_labels:
            if i >= n_features:
                break
        # Get values of the column
        values = df[column].values.reshape(-1,1)
        # Take the scaler of that column
        scaler = scalers['scaler_' + column]
        # Scale values
        scaled_column = scaler.transform(values)
        scaled_column = np.reshape(scaled_column,len(scaled_column))
        # Add the scaled values to the df
        df[column] = scaled_column
        
    print(f' Min values are: ')
    print(df.min())
    print(f' Max values are: ')
    print(df.max())
        
    return df 

In [15]:
# Merge:
train_df = pd.concat([train_df[0], train_df[1], train_df[2], train_df[3], train_df[4]], keys=Municipalities)
test_df = pd.concat([test_df[0], test_df[1], test_df[2], test_df[3], test_df[4]], keys=Municipalities)

In [16]:
feature_range = (-1, 1)


# Scale train:
train_df, scalers = normalize_train_features(train_df, feature_range=feature_range, n_labels=n_labels)
train_df = [train_df[train_df.index.get_level_values(0) == municipality] for municipality in Municipalities]

#print(f'The scalers are: {scalers}')

train_df[0].head()

 Min values are: 
0      -1.0
1      -1.0
2      -1.0
3      -1.0
4      -1.0
       ... 
1022   -1.0
1023   -1.0
0       0.0
1       0.0
2       0.0
Length: 1027, dtype: float64
 Max values are: 
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1022    1.0
1023    1.0
0       1.0
1       1.0
2       1.0
Length: 1027, dtype: float64


0         1         2         3         4         5  \
Medellín 201601  0.472253 -0.239608 -0.177216 -0.384242  0.233995  0.590570   
         201602 -0.431231 -0.094417  0.499012  0.032759 -0.156063  0.056569   
         201603 -0.097089  0.058086 -0.045054 -0.115610 -0.435372  0.238276   
         201604  0.323309 -0.430234  0.733661  0.811450 -0.487762  0.520374   
         201605 -0.142577  0.114798  0.117410  0.094378  0.277193  0.287878   

                        6         7         8         9  ...      1017  \
Medellín 201601 -0.000915 -0.028258  0.224928  0.310899  ... -0.043455   
         201602 -0.437295 -0.614264  0.493422 -0.093184  ... -0.037194   
         201603  0.273789 -0.348902  0.032738  0.390453  ... -0.092698   
         201604  0.082014  0.070235  0.039889  0.016860  ...  0.420393   
         201605 -0.487332 -0.414390  0.007978 -0.397224  ... -0.018164   

                     1018      1019      1020      1021      1022      1023  \
Medellín 201601  0.050324 -0.208524 -0.170808  0.075005 -0.339509 -0.512426   
         201602 -0.622080  0.039078 -0.618765 -0.118612  0.173002  0.092655   
         201603 -0.061219  0.162396  0.000772 -0.736478  0.277905 -0.595254   
         201604 -0.426137  0.561073  0.317632 -0.522704  0.026980 -0.596742   
         201605 -0.209570 -0.065425 -0.083656 -0.903213  0.028389  0.182394   

                 0  1  2  
Medellín 201601  1  0  0  
         201602  0  1  0  
         201603  0  0  1  
         201604  1  0  0  
         201605  1  0  0  

[5 rows x 1027 columns]

In [17]:
feature_range = (-1, 1)

# Scale test:
test_df = normalize_test_features(test_df, scalers=scalers, n_labels=n_labels)
test_df = [test_df[test_df.index.get_level_values(0) == municipality] for municipality in Municipalities]

test_df[0].head()

 Min values are: 
0      -0.841098
1      -0.733276
2      -0.888434
3      -1.005839
4      -0.840073
          ...   
1022   -0.668194
1023   -0.994664
0       0.000000
1       0.000000
2       0.000000
Length: 1027, dtype: float64
 Max values are: 
0       0.678767
1       0.737547
2       1.307462
3       0.774928
4       0.573958
          ...   
1022    0.951445
1023    1.083882
0       1.000000
1       1.000000
2       1.000000
Length: 1027, dtype: float64


0         1         2         3         4         5  \
Medellín 201821 -0.030583  0.387981 -0.084118  0.003712 -0.047331  0.189697   
         201822  0.536719  0.737547 -0.163412 -0.248850  0.059970 -0.399107   
         201823  0.098897 -0.226690  0.378888  0.290940 -0.006128 -0.057011   
         201824  0.236424  0.256422 -0.385421  0.449706 -0.397187 -0.408879   
         201825  0.026130 -0.576934  0.448658 -0.370407  0.401051 -0.053792   

                        6         7         8         9  ...      1017  \
Medellín 201821  0.025916 -0.026069 -0.577718  0.060066  ... -0.455406   
         201822  0.423894 -0.151604  0.150001  0.427886  ...  0.197827   
         201823 -0.015477 -0.427686  0.265408  0.424797  ... -0.158766   
         201824  0.138405 -0.461034  0.269291  0.160264  ...  0.013016   
         201825  0.079800 -0.059748 -0.046672 -0.213356  ... -0.067556   

                     1018      1019      1020      1021      1022      1023  \
Medellín 201821  0.057337  0.325207  0.072952  0.032879  0.227313  0.240158   
         201822 -0.555976  0.024282 -0.217827  0.030148  0.110685  0.089387   
         201823  0.094504  0.453196 -0.716627 -0.607469 -0.175330 -0.166873   
         201824 -0.462455  0.185700  0.192366 -0.032571  0.452517 -0.088618   
         201825  0.073248 -0.301006  0.083398  0.132447  0.358438  0.369217   

                 0  1  2  
Medellín 201821  1  0  0  
         201822  1  0  0  
         201823  1  0  0  
         201824  1  0  0  
         201825  1  0  0  

[5 rows x 1027 columns]

### Prepare data for time series supervised learning (function to create sliding window)

In [18]:
# prepare data for time series

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, no_autoregressive=None):
    
    if no_autoregressive:
        n_in = n_in - 1
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        
        if no_autoregressive:
            cols.append(df.shift(i).iloc[:,:-3])
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars-3)]
        else:
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
            
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [19]:
# length of window
days = 10

# length of window
days = 10
no_autoregressive = True

# frame as supervised learning
train = [series_to_supervised(df, n_in=days, no_autoregressive=no_autoregressive) for df in train_df]
test = [series_to_supervised(df, n_in=days, no_autoregressive=no_autoregressive) for df in test_df]

DataFrame(train[0])

var1(t-9)  var2(t-9)  var3(t-9)  var4(t-9)  var5(t-9)  \
Medellín 201610   0.472253  -0.239608  -0.177216  -0.384242   0.233995   
         201611  -0.431231  -0.094417   0.499012   0.032759  -0.156063   
         201612  -0.097089   0.058086  -0.045054  -0.115610  -0.435372   
         201613   0.323309  -0.430234   0.733661   0.811450  -0.487762   
         201614  -0.142577   0.114798   0.117410   0.094378   0.277193   
...                    ...        ...        ...        ...        ...   
         201816   0.607659  -0.174419  -0.099314  -0.018419  -0.799947   
         201817   0.342440   0.072566   0.255994   0.073019  -0.515819   
         201818   0.021197  -0.482701  -0.037219  -0.238611  -0.234602   
         201819   0.369665  -0.027732   0.425411   0.032442   0.062702   
         201820   0.113725   0.046942   0.124195   0.325223   0.437056   

                 var6(t-9)  var7(t-9)  var8(t-9)  var9(t-9)  var10(t-9)  ...  \
Medellín 201610   0.590570  -0.000915  -0.028258   0.224928    0.310899  ...   
         201611   0.056569  -0.437295  -0.614264   0.493422   -0.093184  ...   
         201612   0.238276   0.273789  -0.348902   0.032738    0.390453  ...   
         201613   0.520374   0.082014   0.070235   0.039889    0.016860  ...   
         201614   0.287878  -0.487332  -0.414390   0.007978   -0.397224  ...   
...                    ...        ...        ...        ...         ...  ...   
         201816   0.623936  -0.348338  -0.071399   0.053836   -0.124933  ...   
         201817  -0.573223  -0.341847   0.369359   0.386798    0.510768  ...   
         201818  -0.045456  -0.580871   0.270278  -0.246214    0.559150  ...   
         201819  -0.227167  -0.303496  -0.196258   0.279451    0.332816  ...   
         201820   0.363185  -0.241430  -0.314433   0.446638   -0.009586  ...   

                 var1018(t)  var1019(t)  var1020(t)  var1021(t)  var1022(t)  \
Medellín 201610    0.356531   -0.144988   -0.690476   -0.090274   -0.013287   
         201611    0.159270   -0.094039    0.336612   -0.221946    0.231081   
         201612    0.425101   -0.139095   -0.354131    0.013865    0.048719   
         201613   -0.040438    0.455289    0.181081   -0.017712   -0.479298   
         201614    0.258198   -0.393841    0.152128    0.372799   -0.017893   
...                     ...         ...         ...         ...         ...   
         201816    0.307833    0.255169    0.425933   -0.311593   -0.326603   
         201817   -0.673182   -0.203357    0.386560    0.189656   -0.319556   
         201818   -0.065025   -0.056931    0.069384   -0.118911    0.321721   
         201819    0.236831   -0.607852    0.125837   -0.118978    0.573862   
         201820    0.415370   -0.351268    0.003531   -0.292212   -0.113917   

                 var1023(t)  var1024(t)  var1025(t)  var1026(t)  var1027(t)  
Medellín 201610    0.093035    0.019838           0           0           1  
         201611   -0.050993   -0.021817           0           1           0  
         201612    0.248950   -0.495737           0           1           0  
         201613    0.062799   -0.809872           0           1           0  
         201614   -0.007668    0.823465           0           0           1  
...                     ...         ...         ...         ...         ...  
         201816    0.377827   -0.162433           1           0           0  
         201817   -0.053125    0.661001           1           0           0  
         201818   -0.117333   -0.264759           1           0           0  
         201819    0.130147    0.614479           1           0           0  
         201820    0.103312   -0.400121           1           0           0  

[115 rows x 10243 columns]

### Merge train data

In [20]:
# Merge:
train = pd.concat([train[0], train[1], train[2], train[3], train[4]], keys=Municipalities)
test = pd.concat([test[0], test[1], test[2], test[3], test[4]], keys=Municipalities)

In [21]:
train.shape

(575, 10243)

### Features and Labels Set

In [22]:
def features_labels_set(timeseries_data, original_df, n_labels):
    
    """ Features """
    # We define the number of features as (features and labels)
    n_features = original_df.shape[1]

    # The features to train the model will be all except the values of the actual week 
    # We can't use other variables in week t because whe need to resample a a 3D Array
    features_set = DataFrame(timeseries_data.values[:,:-n_labels])
    # Convert pandas data frame to np.array to reshape as 3D Array
    features_set = features_set.to_numpy()
    print(f'The shape of the features is {features_set.shape}')
    
    """ Labels """
    # We will use labels in last week 
    labels_set = DataFrame(timeseries_data.values[:,-n_labels:])
    # Convert pandas data frame to np.array
    labels_set = labels_set.to_numpy()
    print(f'The shape of the labels is {labels_set.shape}')
    
    return features_set, labels_set, n_features

In [23]:
# Train features and labels set
print('Train:')
train_X, train_y, n_features = features_labels_set(timeseries_data=train, original_df=dengue_df[0], n_labels=n_labels)

# Test features and labels set
print('Test:')
test_X, test_y, n_features = features_labels_set(timeseries_data=test, original_df=dengue_df[0], n_labels=n_labels)

Train:
The shape of the features is (575, 10240)
The shape of the labels is (575, 3)
Test:
The shape of the features is (115, 10240)
The shape of the labels is (115, 3)


# Modeling

In [24]:
def reshape_tensor(train_X, test_X, n_features):
    print('The initial shapes are:')
    print(f'The train shape is {train_X.shape}')
    print(f'The test shape is {test_X.shape}')
    
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], days, n_features-n_labels))
    test_X = test_X.reshape((test_X.shape[0], days, n_features-n_labels))
    
    print('-----------------------')
    print('The Final shapes are:')
    print(f'The train shape is {train_X.shape}')
    print(f'The test shape is {test_X.shape}')
    
    return train_X, test_X

In [25]:
# reshape input to be 3D [samples, timesteps, features]
train_X, test_X = reshape_tensor(train_X, test_X, n_features)

The initial shapes are:
The train shape is (575, 10240)
The test shape is (115, 10240)
-----------------------
The Final shapes are:
The train shape is (575, 10, 1024)
The test shape is (115, 10, 1024)


# Define the Model

In [26]:
# Set Seed
#tf.random.set_seed(0)

def create_model():
    # design network
    model = Sequential()
    model.add(LSTM(120, dropout=0.1, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
    model.add(LSTM(240, dropout=0.1, input_shape=(train_X.shape[1], 120)))
    model.add(Dense(60))
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model:
    opt = keras.optimizers.Adam()
    metrics = [
        tf.keras.metrics.AUC(name='auc', multi_label=True, num_labels=3),
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.CategoricalCrossentropy(name='entropy')
    ]
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=metrics)

    return model

### Train the model

In [27]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, 
        verbose=1, mode='auto', restore_best_weights=True)

In [28]:
### Imbalanced data
n_zeros = train_y[:,0].sum()
n_ones = train_y[:,1].sum()
n_twos = train_y[:,2].sum()
n_total = n_zeros + n_ones + n_twos

weights = {0: n_total/n_zeros, 1: n_total/n_ones, 2: n_total/n_twos}
print(f'zeros: {n_zeros}, ones: {n_ones}, twos: {n_twos}, total: {n_total}')
weights

zeros: 434.0, ones: 62.0, twos: 79.0, total: 575.0


{0: 1.3248847926267282, 1: 9.274193548387096, 2: 7.2784810126582276}

In [29]:
# fit network
def train_model(model, monitor, weights, plot=None, epochs=20):
    if monitor and weights:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor], class_weight=weights)
    elif monitor:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor])
    elif weights:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, class_weight=weights)
    else:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    
    if plot:
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.legend()
        plt.show()
        

# Test the model

# AUC

In [30]:
# You can also evaluate or predict on a dataset.
def evaluate(model, verbose = None):
    if verbose:
        print('Evaluate: ')
    result = model.evaluate(test_X, test_y)
    stored_results = {}
    for i, metric in enumerate(model.metrics_names):
        stored_results[metric] = result[i]
        if verbose:
            print(f'{metric}: {result[i]}')
    return stored_results

# Calculate Mean and SD

In [31]:
def calculate_mean_std(weights):
    
    metrics = {
        "auc": [],
        "acc": [],
        "entropy": []
    }
    
    for i in range(3):
        model = create_model()
        train_model(model=model, monitor=monitor, weights=weights)
        stored_results = evaluate(model=model)
        
        for key in metrics.keys():
            metrics[key].append(stored_results[key])
            
    for key in metrics.keys():
        results = metrics[key]
        print(key, f": average={np.average(results):.3f}, std={np.std(results):.3f}")


In [32]:
#calculate_mean_std(weights=None)
calculate_mean_std(weights=weights)

Epoch 1/20
36/36 - 4s - loss: 3.4760 - auc: 0.4678 - acc: 0.3287 - entropy: 1.1072 - val_loss: 1.1247 - val_auc: 0.5447 - val_acc: 0.2522 - val_entropy: 1.1247
Epoch 2/20
36/36 - 1s - loss: 2.8834 - auc: 0.7382 - acc: 0.4939 - entropy: 0.9546 - val_loss: 1.1707 - val_auc: 0.5705 - val_acc: 0.3826 - val_entropy: 1.1707
Epoch 3/20
36/36 - 1s - loss: 2.6227 - auc: 0.8156 - acc: 0.6974 - entropy: 0.7814 - val_loss: 1.0202 - val_auc: 0.5803 - val_acc: 0.4783 - val_entropy: 1.0202
Epoch 4/20
36/36 - 1s - loss: 2.3448 - auc: 0.8545 - acc: 0.6800 - entropy: 0.7684 - val_loss: 1.0674 - val_auc: 0.4958 - val_acc: 0.6174 - val_entropy: 1.0674
Epoch 5/20
36/36 - 1s - loss: 1.9885 - auc: 0.8734 - acc: 0.6557 - entropy: 0.7765 - val_loss: 1.7635 - val_auc: 0.4526 - val_acc: 0.6783 - val_entropy: 1.7635
Epoch 6/20
36/36 - 1s - loss: 1.6715 - auc: 0.9001 - acc: 0.7061 - entropy: 0.6784 - val_loss: 1.5379 - val_auc: 0.5020 - val_acc: 0.6783 - val_entropy: 1.5379
Epoch 7/20
36/36 - 1s - loss: 1.7450 - a

In [33]:
#model = create_model()
#train_model(model=model, monitor=monitor, weights=weights)
#stored_results = evaluate(model=model)
#print(stored_results)